In [1]:
import os

import pandas as pd
import numpy as np
import utils.dhs_config as dhs_config
import utils.dhs_utils as dhs

In [2]:
# Main folder with respect from the scripts
MAIN = "../"

# Data folder
DATA = os.path.join(MAIN, "data")

# Raw data folder (where .sav files are)
RAW_DATA = os.path.join(DATA, "raw")

# Transformed data folder
TRS_DATA = os.path.join(DATA, "transformed")

column_variables, column_others = dhs_config.get_columns()

if not(os.path.exists(os.path.join(TRS_DATA, 'df_all.csv'))):
    df_all, couple_all = dhs.get_all_data(RAW_DATA,
                                          column_variables + column_others)
    
    # In year 1994 and 1996 there is some change in the psy question
    # so we skip thoses and also only take the variables of interest
    df_reduce = df_all[~df_all.year.isin([1994, 1996])][column_variables]
    couple_reduce = couple_all[~couple_all.year.isin([1994, 1996])]
    couple_clean = couple_reduce[couple_reduce.married == 1]

    # we clean the data and drop schooling as we did not have the time to clean it
    teg_dict, spaar_dict = dhs_config.get_dict()
    df_clean = df_reduce.replace(teg_dict).replace(spaar_dict).drop('scholing', axis=1)
    
    df_all.to_csv(os.path.join(TRS_DATA, 'df_all.csv'))
    couple_all.to_csv(os.path.join(TRS_DATA, 'couple_all.csv'))
    df_clean.to_csv(os.path.join(TRS_DATA, 'df_clean.csv'))
    couple_clean.to_csv(os.path.join(TRS_DATA, 'couple_clean.csv'))

else:
    df_clean = pd.read_csv(os.path.join(TRS_DATA, 'df_clean.csv'), index_col=0)
    couple_clean = pd.read_csv(os.path.join(TRS_DATA, 'couple_clean.csv'), index_col=0)

In [3]:
df_clean.sample(5)

,gez1,gez3,bmi,spaar1,spaar2,spaar3,spaar4,spaar5,spaar6,teg1,...,teg7,teg8,teg9,teg10,teg11,teg12,teg13,teg14,teg15,teg16
199301880001,187.0,Excellent,24.307244,4,3,5,4,3,4,4,...,5,2,6,2,2,3,4,6,6,2
199523920001,185.0,excellent,23.959094,4,4,3,4,4,4,5,...,4,6,3,5,3,4,6,5,3,5
2002132200002,166.0,fair,26.854406,2,2,2,7,1,1,4,...,4,3,4,3,3,6,3,3,5,4
199524380002,177.0,fair,28.727377,6,2,5,6,6,4,5,...,5,3,5,3,4,3,5,5,4,4
199532770002,173.0,good,26.729927,6,7,1,6,6,5,5,...,4,6,2,3,4,5,3,4,6,4


In [4]:
couple_clean.sample(5)

,part1,part2,married,year
683,199322200001,199322200002,1,1993
120,2001133500002,2001133500001,1,2001
499,199315750001,199315750002,1,1993
318,199309710001,199309710002,1,1993
320,199309800002,199309800001,1,1993
